# Baseline

In [8]:
import torch
from PIL import Image
import torchvision.transforms as T
from torch import nn
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
input_dir = "/kaggle/input/blood-vessel-segmentation/"
train_dir = input_dir + "train"

## Load data

**Datasets**

- kidney_1_dense	
    - images
    - labels
- kidney_1_voi  
    - images
    - labels
- kidney_2	
    - images
    - labels
- kidney_3_dense	
    - labels
- kidney_3_sparse
    - images
    - labels

### ¿Do I use one dataset or all of them? ¿How do I merge all the datasets?

I think i can merge the datasets by just merging the images and labels paths. But for now I will just use the `kidney_1_dense` dataset.

### Dataset

In [5]:

class KidneyDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, msk_paths, transform=None, target_transform=None):
        self.img_paths  = img_paths
        self.msk_paths  = msk_paths
        self.transform = transform
        self.target_transform = target_transform
        assert len(self.img_paths) == len(self.msk_paths)
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        msk_path = self.msk_paths[idx]

        img = Image.open(img_path)
        msk = Image.open(msk_path)

        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            msk = self.target_transform(msk)

        return img, msk

In [6]:
k1d_path = "kidney_1_dense"
k1d_imgs_paths = sorted([f"{train_dir}/{k1d_path}/images/{id}" for id in os.listdir(f"{train_dir}/{k1d_path}/images")])
k1d_msks_paths = sorted([f"{train_dir}/{k1d_path}/labels/{id}" for id in os.listdir(f"{train_dir}/{k1d_path}/labels")])

k1d_ds = KidneyDataset(k1d_imgs_paths, k1d_msks_paths, transform=T.ToTensor(), target_transform=T.PILToTensor())

In [7]:
x, y = k1d_ds[0]
print(x)
print(y)
print(x.shape)
print(y.shape)

tensor([[[22229, 21842, 22422,  ..., 20789, 20733, 20711],
         [22458, 22184, 22584,  ..., 21060, 20893, 20920],
         [22308, 22078, 22214,  ..., 21242, 20942, 20640],
         ...,
         [21848, 21609, 22175,  ..., 21029, 20890, 21145],
         [21573, 21569, 22062,  ..., 20899, 20983, 20793],
         [21617, 21943, 22238,  ..., 21113, 20947, 20557]]], dtype=torch.int16)
tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)
torch.Size([1, 1303, 912])
torch.Size([1, 1303, 912])


## Define model

In [10]:
class SegmentationModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SegmentationModel, self).__init__()

        # Define the encoder - a series of convolutional layers
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Define the decoder - a series of transposed convolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, num_classes, kernel_size=3, padding=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the model with the desired input channels and number of classes
model = SegmentationModel(in_channels=3, num_classes=2)

# Use the model for a batch of input images
input_batch = torch.rand(1, 3, 256, 256)
output = model(input_batch)

In [11]:
print(output)
print(output.shape)

tensor([[[[ 6.2274e-02,  3.7559e-02,  3.8789e-02,  ...,  3.7079e-02,
            4.2650e-02,  2.8734e-02],
          [ 3.7905e-02,  5.8133e-02,  3.6838e-02,  ...,  5.7836e-02,
            3.8458e-02,  4.3214e-02],
          [ 6.7339e-02,  4.0514e-02,  4.8800e-02,  ...,  4.0044e-02,
            5.5550e-02,  2.5583e-02],
          ...,
          [ 4.5950e-02,  4.7073e-02,  5.4629e-02,  ...,  5.2956e-02,
            3.7570e-02,  4.0345e-02],
          [ 6.8837e-02,  4.0746e-02,  5.2075e-02,  ...,  3.9960e-02,
            5.2678e-02,  2.8405e-02],
          [ 4.5970e-02,  4.4272e-02,  3.3345e-02,  ...,  4.3463e-02,
            4.5872e-02,  3.4378e-02]],

         [[ 2.4114e-05, -3.1747e-03, -6.8101e-03,  ..., -1.1042e-03,
           -9.0612e-03, -1.7117e-02],
          [-1.9466e-02, -3.1621e-02, -2.9733e-02,  ..., -3.5240e-02,
           -1.9901e-02, -1.9310e-02],
          [ 7.2741e-04, -1.0017e-02, -2.8085e-02,  ..., -1.1614e-02,
           -2.6728e-02, -2.3124e-02],
          ...,
     

## Define evaluation metric

In [ ]:
class IouMetric:
    def __init__(self, num_classes: int, int2str: dict, ignore_index: int = 255, prefix="train"):
        """
        Args:
            num_classes: number of classes
            int2str: dictionary mapping class index to class name
            ignore_index: index to ignore in the metric calculations
            prefix: prefix to use for logging
        """
        self.area_intersect = torch.zeros(num_classes)
        self.area_label = torch.zeros(num_classes)
        self.area_pred = torch.zeros(num_classes)
        self.num_classes = num_classes
        self.int2str = int2str
        self.ignore_index = ignore_index
        self.prefix = prefix

    def process(self, preds, labels):
        mask = labels != self.ignore_index
        preds = preds[mask]
        labels = labels[mask]

        # compute area of intersection, label and prediction
        intersect = preds[preds == labels]
        area_intersect = torch.histc(intersect.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)
        area_label = torch.histc(labels.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)
        area_pred = torch.histc(preds.float(), bins=self.num_classes, min=0, max=self.num_classes - 1)

        # update results
        self.area_intersect += area_intersect.cpu()
        self.area_label += area_label.cpu()
        self.area_pred += area_pred.cpu()

    def compute(self) -> dict:
        iou = self.area_intersect / (self.area_label + self.area_pred - self.area_intersect)
        if 0 <= self.ignore_index <= self.num_classes - 1:
            iou[self.ignore_index] = torch.nan
        mean_iou = torch.nanmean(iou)

        metrics = {
            f"{self.prefix}/iou_{self.int2str[idx]}": iou[idx].item()
            for idx in range(len(iou))
            if idx != self.ignore_index
        }
        metrics[f"{self.prefix}/mean_iou"] = mean_iou.item()

        return metrics

    def reset(self):
        self.area_intersect.zero_()
        self.area_label.zero_()
        self.area_pred.zero_()


## Define loss function


In [9]:
loss_fn = nn.BCEWithLogitsLoss()

## Train model

### Optimizer and scheduler

### Train method

### Evaluation method

## Make submisssion